# Price recommendation system for OLX cars announcements
The aim of this work is to create machine learning model to suggest car price in announcements. 
The recommendation will help users based on information about vehicle to establish initial price.
The source of the data is listed below:

https://www.kaggle.com/datasets/abdullahkhanuet22/olx-cars-dataset

## Load packages and data

In [ ]:
import pandas as pd 
import numpy as np 
import os 
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from functions import kaggle_download, barplot_share, info_splitter, calc_split_summary, bubble_plot
import lightgbm as lgb 
import optuna
seed=123

In [ ]:
optuna.create_study().optimize(n)

In [ ]:
kaggle_download(user='abdullahkhanuet22',dataset='olx-cars-dataset')

In [ ]:
data = pd.read_csv('OLX_cars_dataset.csv')

## Data understanding
The first step is to take a brief look at data and evaluate it.

In [ ]:
data.info()

- Dataset cointains 8721 records which is not bad for the excercises purpose. 
- It doesn't contain any blanks. 
- Most of the variables are strings/ categorical.
- Only Price (which is target to the model), year of production and kilometers driven (KM's driven) are integer.
- It would be hard to use model with year as a variable, that's why new variable 'Age' of the car will be calculated (in years). There is no information what was the date of collecting the data, so it is calculated based on the most current year in the dataset which is 2024.

In [ ]:
data['Age'] = data['Year'].max() - data['Year']

## Categorical variables analysis
The most typical categorical features are name / model of the car and make.

Car name has a lot of different values, so it is not taking under consideration for now.

The first variable to analyze is make.

In [ ]:
data['Car Name'].value_counts()

### 'Make' variable

In [ ]:
data['Make'].value_counts()

There are 11 different makes, some of them need to be marged because of being sparse.
To merged that correctly - similarity analysis is conducted below. 

In [ ]:
bubble_plot(df=data,x='Price',y='Age',ids_col = 'Ad ID',groupper = 'Make',list_to_change_location=['Chevrolet'],agg_func = np.mean)

In [ ]:
prices = data.groupby('Make').agg({'Price':np.mean,'Age': np.mean ,'Ad ID': np.size}).rename({'Ad ID': 'No of ads'},axis=1).reset_index()
prices['Age'] = np.round(prices['Age'])
prices

In [ ]:
text_pos = ['top center' if i !='Chevrolet' else 'bottom center' for i in prices['Make'] ]
fig = px.scatter(prices, x='Price',y='Age',size='No of ads',text='Make')
fig.update_traces(textposition=text_pos)
fig.update_layout(
    title_text="Cars' Price and Age by make"
)
fig.show()

In [ ]:
prices = data.groupby('Make').agg({'Price':np.median,'Age': np.median ,'Ad ID': np.size}).rename({'Ad ID': 'No of ads'},axis=1).reset_index()
prices['Age'] = np.round(prices['Age'])
prices['Share of make'] =  np.round(prices['No of ads']/ prices['No of ads'].sum(),3)
text_pos = ['top center' if i !='Chevrolet' else 'bottom center' for i in prices['Make'] ]
fig = px.scatter(prices, x='Price',y='Age',size='No of ads',text='Make')
fig.update_traces(textposition=text_pos)
fig.update_layout(
    title_text="Cars' Price and Age by make"
)
fig.show()

In [ ]:
barplot_share(data, 'Make')

Based on above plots there is a need to merge some make to have bigger group.
There will be two methods provided:
1. The makes with number of ads lower than 3% will be merged to one group.
2. The makes with share <3% will be group according to average price and age:
    - Mercedes and Changan will be group with Toyota (the closest average price).
    - Chevrolet, Hyundai and FAW will be grouped with Suzuki (the closest average price)
    - Mitshubishi and KIA will be merged to one group (the closest average price and age).


In [ ]:
makes_less_3_prct = prices.loc[prices['Share of make']<0.03,'Make'].values
makes_less_3_prct

In [ ]:
data['make_new_1'] = data.loc[:,'Make'].apply(lambda x: 'Others' if x in makes_less_3_prct  else x)

In [ ]:
barplot_share(data, 'make_new_1')

##### Version 2

In [ ]:
group_1 = ['Mercedes', 'Changan','Toyota']
group_2 = ['Chevrolet', 'Hyundai',  'FAW', 'Suzuki']
group_3 = ['Mitsubishi', 'KIA']

#data['make_new_2'] = np.where(data['Make'].isin(group_1),'Merc_chang_toy',np.where(data['Make'].isin(group_2), ))

In [ ]:
data['make_new_2'] = np.select([data['Make'].isin(group_1),
                                data['Make'].isin(group_2),
                                data['Make'].isin(group_3)],['Merc_chang_toy','chev_huy_faw_suz','mits_kia'], default=data['Make'])

In [ ]:
barplot_share(data, 'make_new_2')

At first step let's model optimization allow to choose better variable

### Categorical Features

In [ ]:
columns = ['Fuel','Registration city',
'Car documents',
'Assembly',
'Transmission' ,
'Condition',
'Seller Location' ]

Based on below summary variables Fuel, Assembly, Registration city and Transmission can be used.

In [ ]:
for col in columns:
    print(data[col].value_counts())
    print('################\n')

In [ ]:
columns = ['Fuel',
'Assembly',
'Transmission' ,
]

In [ ]:
for col in columns:
    print(data[col].value_counts())
    print('################\n')

For fuel variable Only 'petrol' value is significant high, so the variable will be changed to true/false variable (is petrol?)

In [ ]:
data['is_petrol'] = data['Fuel'] =='Petrol'

In [ ]:
barplot_share(data, 'is_petrol')

Only 2% of cases are not Petrol fuel, so there is a risk to use this variable in the model.

#### 
Make - to leave for later - many values

In [ ]:
reg_city = data['Registration city'].value_counts() / data.shape[0] <0.03
big_cities=  reg_city[reg_city== False].index
data['registration_city'] = np.where(data['Registration city'].isin(big_cities),data['Registration city'],'Other')

In [ ]:
barplot_share(data, 'registration_city')

In [ ]:
barplot_share(data, 'Assembly')

In [ ]:
barplot_share(data, 'Transmission')

### Check categorical variables relation with price
Correlation between categorical and numeric variable can be confusing. That is why in the first step average price within groups will be checked.

In [ ]:
columns = ['make_new_1', 'make_new_2' , 'is_petrol', 'Assembly', 'registration_city' , 'Transmission']

In [ ]:
for col in columns:
    df_to_plot = data[[col,'Price']].groupby(col).mean().reset_index().sort_values(by='Price',ascending=False)
    fig = px.bar(data_frame=df_to_plot, x=col, y='Price',title=f'Average Price by {col}')
    fig.show()

All of the variables have impact on price. To make a second check distribution plot will be made for variables with two values.

In [ ]:
columns_short = [ 'is_petrol', 'Assembly', 'Transmission']

In [ ]:
for col in columns_short:
    fig = px.histogram(data_frame=data,x='Price',color=col, marginal="box",histnorm='probability', title=f'Price Histogram by {col}')
    fig.update_traces(opacity=0.75)
    fig.update_layout(barmode='overlay')
    fig.show()

Above plots show histograms and box plots by subgroups (defined by chosen 0/1 variables). Box plots highlight differences of average and quartiles in each group. The minimum and maximum are the closest for assembly. 
Each value hase higher predictive power if histograms are more separate which can be seen the most in transmition variable.

#### Car feaures
There is one more, interesting variable - car features. Unlikely, all features are stored in one column. Split to separate columns will be provided.

In [ ]:
data, all_features = info_splitter(data, 'Car Features',split = ', ')

In [ ]:
for feat in all_features:
    print(data[feat].value_counts())
    print('##############\n')

In [ ]:
all_diff = []
min_no_of_rec = []
for feat in all_features:
    agg = data[[feat,'Price','Ad ID']].groupby(feat).agg({'Price':np.mean, 'Ad ID':np.size}).rename({'Ad ID': 'no of records'}, axis=1)
    min_no_of_rec.append(agg['no of records'].min())
    diff = round(agg.loc[1,'Price'] / agg.loc[0,'Price'] * 100)
    all_diff.append(diff)
    print(agg)
    print(f" Difference between value 1 and 0 equals to: {diff}%")
    print('##############\n')

In [ ]:
all_diff_df = pd.DataFrame(data=all_diff, index = all_features, columns=['diff %'])
all_diff_df['smaller cat cases'] = min_no_of_rec
all_diff_df.sort_values(by='diff %')

Minimum sample of smaller category in each car features is grater than 500, so all can be used. What is more all of them differentiate average price. Before final decision of which of them should be used in the model, median will be evaluated.

In [ ]:
all_diff = []
min_no_of_rec = []
for feat in all_features:
    agg = data[[feat,'Price','Ad ID']].groupby(feat).agg({'Price':np.median, 'Ad ID':np.size}).rename({'Ad ID': 'no of records'}, axis=1)
    min_no_of_rec.append(agg['no of records'].min())
    diff = round(agg.loc[1,'Price'] / agg.loc[0,'Price'] * 100)
    all_diff.append(diff)
    print(agg)
    print(f" Difference between value 1 and 0 equals to: {diff}%")
    print('##############\n')

In [ ]:
all_diff_df = pd.DataFrame(data=all_diff, index = all_features, columns=['diff %'])
all_diff_df['smaller cat cases'] = min_no_of_rec
all_diff_df.sort_values(by='diff %')

The differences in median are slighlty higher than for average. In the first step all of these features will be used and after first optimization the worst variables will be excluded.

### Numerical features

In [ ]:
numerical_features = ["KM's driven","Age","Price"]

In [ ]:
sns.heatmap(data[numerical_features].corr(),annot=True)
plt.title('Correlation matrix for numerical features')
plt.show()

In [ ]:
sns.pairplot(data[numerical_features])
plt.show()

Above plots show the logical relation between age of the car and price. The kilometers made by the car has much lower impact on price. Scatter plot for 8000 cases can be confusing and is only additional information - small trend in both - Age and KM's driven and Price can be seen. Histograms indicate good number of cases for all ranges of the variables. Lack of cases above 25 years and 300 000 km should be higlihted. Exact number will be checked - it is important to not use the model for cases above that values, especially for Age. 

In [ ]:
data[numerical_features].describe()

Maximum value for Age is equal to 35 and for KM's driven approximetaly 530 000. However there is no enough data point around that points to use them in the final solution. Additional quantiles will be checked.

In [ ]:
data[numerical_features].quantile([0.95,0.98,0.99])

The recommender system shouldn't be used for cars with Age>23 (0.98 quantile) and KM's driven >250 000.

In [ ]:
data.to_csv('olx_cars_adjusted.csv')